In [2]:
from PIL import Image
import numpy as np
from utils import mask_utils, box_utils
from utils.object_detection import visualization_utils
from pycocotools import mask as mask_api
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
from tqdm.notebook import tqdm
import os 
import dotenv
dotenv.load_dotenv()

%matplotlib inline 

model = tf.saved_model.load(os.getenv('MODEL_DIR'))

def infer(image_pattern):

    label_map_dict = {}
    with open('datasets/fashionpedia_label_map.csv', 'r') as csv_file:
        reader = csv_file.readlines()
        for row in reader:
            k, v = row.split(':')
            id_index = int(k)
            name = v
            label_map_dict[id_index] = {
                'id': id_index,
                'name': name,
            }
    res = []
    image_files = glob.glob(image_pattern)
    for i, image_file in tqdm(enumerate(image_files), total = len(image_files)):
        print(f'processing image {i}, {image_file}', end = '\t\t\t\t\r')
        
        image = Image.open(image_file)    
        image_array = np.array(image)[:,:,:3] #remove alpha channel
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_array, axis=0), dtype=tf.uint8)
        
        # Perform inference
        
        serving_default_fn = model.signatures['serving_default']
        output_results = serving_default_fn(input_tensor)
            
        image_with_detections_list = []
        
                
        num_detections = int(output_results['num_detections'][0])
        np_boxes = output_results['detection_boxes'][0, :num_detections]
        width, height = image.size
        
        np_image_info = output_results['image_info'][0]
        np_boxes = np_boxes / np.tile(np_image_info[1:2, :], (1, 2))
        ymin, xmin, ymax, xmax = np.split(np_boxes, 4, axis=-1)
        ymin = ymin * height
        ymax = ymax * height
        xmin = xmin * width
        xmax = xmax * width
        
        np_boxes = np.concatenate([ymin, xmin, ymax, xmax], axis=-1)
        np_scores = output_results['detection_scores'][0, :num_detections]
        np_classes = output_results['detection_classes'][0, :num_detections]
        np_classes = np_classes.numpy().astype(np.int32)
        np_attributes = output_results['detection_attributes'][
        0, :num_detections, :]
        
        np_masks = None
        if 'detection_masks' in output_results:
            np_masks = output_results['detection_masks'][0, :num_detections]
            np_masks = mask_utils.paste_instance_masks(
                np_masks, box_utils.yxyx_to_xywh(np_boxes), height, width)
            encoded_masks = [
                mask_api.encode(np.asfortranarray(np_mask))
                for np_mask in list(np_masks)]
        
        np_image = (image_array.reshape(height, width, 3).astype(np.uint8))
        image_with_detections = (
                visualization_utils.visualize_boxes_and_labels_on_image_array(
                    np_image,
                    np_boxes,
                    np_classes,
                    np_scores,
                    label_map_dict,
                    instance_masks=np_masks,
                    use_normalized_coordinates=False,
                    max_boxes_to_draw=15,
                    min_score_thresh=0.3))
        image_with_detections_list.append(image_with_detections)

        res.append({
                'image_file': image_file,
                'boxes': np_boxes,
                'classes': np_classes,
                'scores': np_scores.numpy(),
                'attributes': np_attributes,
                'masks': encoded_masks,
            })
        # plt.imshow(image_with_detections_list[0])
        # plt.show()

    return res

In [3]:
%%time 
output = infer('../../project-cluster-index/lovebonito_tops/top/**/*.jpg')

  0%|          | 0/1026 [00:00<?, ?it/s]

CPU times: total: 18min 53s./project-cluster-index/lovebonito_tops/top\zurie-flare-sleeve-blouse\hy5117-031.jpg				zxu0vpzm.jpg						1_egiun13li3dmefv2.jpg									
Wall time: 13min 55s


In [4]:
np.save('lovebonito_output', (output), allow_pickle=True)

In [ ]:
import pycocotools.mask as mask_util
import numpy as np
import matplotlib.pyplot as plt
import cv2

with open('datasets/fashionpedia_label_map.csv', 'r') as f:
    mapping = f.readlines()

for item_idx in range(len(output)):
    # item_idx=25
    for i in range(len(output[item_idx]['masks'])):    
        # if output[item_idx]['classes'][i] in (1,2,3):
            # 1:shirt, blouse
            # 2:top, t-shirt, sweatshirt
            # 3:sweater
    
            print(mapping[output[item_idx]['classes'][i] - 1], output[item_idx]['boxes'][i],  output[item_idx]['scores'][i])
            fig, ax = plt.subplots(1, 5, figsize=(10,10))
            img = cv2.imread(output[item_idx]['image_file'])
            image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax[0].imshow(image)
            ax[0].set_title('Original')
    
             # Encoded mask string
            encoded_mask_string = output[item_idx]['masks'][i]
            
            # Decode the mask
            decoded_mask = mask_util.decode(encoded_mask_string)
            
            # Visualize the mask        
            ax[1].imshow(decoded_mask)        
            ax[1].set_title('Mask')
    
            masked_img = np.einsum('hwc,hw->hwc', image, decoded_mask)
            masked_img[masked_img == 0] = 255
            ax[2].imshow(masked_img)
            ax[2].set_title('Masked')
    
            #Crop to bounding boxes        
            y1, x1, y2, x2 = np.ceil(output[item_idx]['boxes'][i]).astype(int)
            crop_img = image[y1:y2, x1:x2]
            ax[3].imshow(crop_img)
            ax[3].set_title('Cropped')
    
            crop_masked_img = masked_img[y1:y2, x1:x2]
            crop_masked_img[crop_masked_img == 0] = 255
            ax[4].imshow(crop_masked_img)
            ax[4].set_title('Masked + Cropped')
           
            plt.tight_layout()
            plt.show()
